# Computing a real life shortest path 

<img align='after' width='180' src='https://drive.google.com/uc?export=view&id=19qZe5VIxkIEm7_hYkrvlH142XPBxBaEf'>

---
 > During this course we make use of Jupyter notebooks hosted by [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb). 
  Notebooks deployed on `colab` require neither python nor other dependencies to be installed on your own machine, you only need a browser (preferably `chrome`) and you may also need a google account if you want to execute them. 
 
---

**Notes:** 
 1. This notebook takes some extra trouble to install some packages, due to a more modern version of `numpy` and `matplotlib` being needed than the one installed by `colab`. 
 1. From now on, we prepare our notebooks for `colab` since we assume that in case you prefer to run them locally then by know you know how and where to adapt them.

In [1]:
import subprocess
import sys

class ColabInstaller():

    def __init__(self):
        reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
        self.installed_packages = [r.decode().split('==')[0] for r in reqs.split()]

    def on_colab(self):
        return "google.colab" in sys.modules

    def install(self, package):
        if self.on_colab():
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
            self.installed_packages.append(package)

    def upgrade(self, package):
        if self.on_colab():
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", package])

colab = ColabInstaller()
colab.upgrade('numpy')
colab.upgrade('matplotlib')
colab.install('geopandas')
colab.install('geopy')
colab.install('osmnx')
colab.install('osmnet')
colab.install('pandana')
colab.upgrade('geopy')

# Introduction

Google brought with [maps]( https://www.google.com/maps) the world to our screens, including accurate geocoding and routing for several modalities. 

For the most, the usage of [maps]( https://www.google.com/maps) is interactive. As data and analytics professionals we often need a programmatically support for the services that [maps]( https://www.google.com/maps) offer us. Preferably free.

It also offers a plethora of [development support](https://developers.google.com/), but unfortunately most is paid. That is even more so for [maps]( https://developers.google.com/maps/documentation).

The same Google also offers us [colab]( https://colab.research.google.com/) and that is exactly where we are now. Colab is [free]( https://research.google.com/colaboratory/faq.html) to use, although it includes a paid [pro]( https://colab.research.google.com/signup) version.

This notebook runs on the free version of colab and illustrates free packages on the [python]( https://www.python.org/) ecosystem. 

Alongside with free code the open source community also offers us a lot of free data and free knowledge. This notebook, and presentation, is just a skin-deep introduction. 

## Some heroes 
[Geoff Boeing]( https://geoffboeing.com/about/) is a true leader in demystifying urban data analytics, with a strong emphasis on street networks. His [peer reviewed publications]( https://geoffboeing.com/publications/) are open and accompanied by usable demonstrations using his own [OSMnx]( https://geoffboeing.com/2018/03/osmnx-features-roundup/) package.
Professor [Peter Sanders]( https://algo2.iti.kit.edu/english/sanders.php), see also his [Wikipedia]( https://en.wikipedia.org/wiki/Peter_Sanders_(computer_scientist)) page, has moved his interests to other areas but his [route planning]( http://algo2.iti.kit.edu/routeplanning.php) project shaped the world of truly scalable road routing algorithms. 
From his alumni I distinguish two persons:
 * [Dominik Schultes](http://algo2.iti.kit.edu/schultes/) who won the [DIMACS challenge on shortest paths]( http://www.diag.uniroma1.it//challenge9/data/tiger/) and made it to the [Scientific American top 50]( https://www.scientificamerican.com/article/sciam-50-the-fastest-way/). Before Dominik’s research scalable shortest paths on large national road networks where heuristics, now they are exact and can be computed at world scale. 
 * [Dennis Luxen]( http://algo2.iti.kit.edu/english/luxen.php) for creating https://github.com/Project-OSRM/osrm-backend which offers a free, scalable, implementation of [contraction hierarchies]( https://en.wikipedia.org/wiki/Contraction_hierarchies).
 
Finally, I mention [Fletcher Foti]( https://fletcherfoti.weebly.com/) who gave us [pandana]( http://udst.github.io/pandana/).
 


Whenever we need more packages, we just install those into our session.

## Geocoding

The world is mapped with the [geographic coordinate system](https://en.wikipedia.org/wiki/Geographic_coordinate_system) but we have difficulties remembering [latitudes]( https://en.wikipedia.org/wiki/Latitude) and [longitudes]( https://en.wikipedia.org/wiki/Longitude).

We learn and remember the world better from addresses. 


In [6]:
import geopy
    
def FreeLocator():
    return geopy.Photon(user_agent='myGeocoder')

Amsterdam = FreeLocator().geocode('Amsterdam,NL')

# Visualization

In [7]:
import folium
Map = folium.Map(location=(Amsterdam.latitude,Amsterdam.longitude), zoom_start=12)
Map

In [8]:
def locate_geopy(description):
    location = FreeLocator().geocode(description)
    if location is not None:
        return location.latitude, location.longitude
    return None, None

In [9]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format

data = {'address': [ 'Centraal Station',
                     'Amsterdam Business School',
                     'Artis',
                     'Arena',
                     'Ziggo Dome' ], 
        'color'  : [ 'blue',
                     'black',                   
                     'green',
                     'red',
                     'purple' ]}
# Create DataFrame.
df = pd.DataFrame(data)
df['city']    = 'Amsterdam'
df['country'] = 'NL'
df

,address,color,city,country
0,Centraal Station,blue,Amsterdam,NL
1,Amsterdam Business School,black,Amsterdam,NL
2,Artis,green,Amsterdam,NL
3,Arena,red,Amsterdam,NL
4,Ziggo Dome,purple,Amsterdam,NL


In [10]:
locations = [ locate_geopy(','.join(row[['address','city','country']])) for _, row in df.iterrows() ]
df['lat'] = [ loc[0] for loc in locations ]
df['lon'] = [ loc[1] for loc in locations ]
df

,address,color,city,country,lat,lon
0,Centraal Station,blue,Amsterdam,NL,52.378901,4.900581
1,Amsterdam Business School,black,Amsterdam,NL,52.365107,4.911718
2,Artis,green,Amsterdam,NL,52.415605,4.837026
3,Arena,red,Amsterdam,NL,52.315990,4.942931
4,Ziggo Dome,purple,Amsterdam,NL,52.313790,4.938438


In [11]:
for _, row in df.iterrows():
    folium.Marker((row.lat,row.lon),icon=folium.Icon(color=row.color), tooltip=row.address).add_to(Map)
Map

In [12]:
import osmnx as ox
import networkx as nx
from IPython.display import display

ox.config(log_console=True, use_cache=True)

In [13]:
%%time 
G_walk = ox.graph_from_place('Amsterdam, NL', network_type='walk')

CPU times: user 1min 14s, sys: 1.38 s, total: 1min 16s
Wall time: 2min 29s


In [ ]:
print( G_walk.number_of_nodes(), G_walk.number_of_edges() )

35218 99120


In [ ]:
df['osmnx'] = ox.distance.nearest_nodes(G_walk,df.lon,df.lat)
df

,address,color,city,country,lat,lon,osmnx
0,Centraal Station,blue,Amsterdam,NL,52.378901,4.900581,5629072001
1,Amsterdam Business School,black,Amsterdam,NL,52.365107,4.911718,46356661
2,Artis,green,Amsterdam,NL,52.415605,4.837026,8116399477
3,Arena,red,Amsterdam,NL,52.315990,4.942931,4622542635
4,Ziggo Dome,purple,Amsterdam,NL,52.313790,4.938438,1925143759


In [ ]:
%time route = nx.shortest_path(G_walk,df.iloc[0].osmnx,df.iloc[1].osmnx,weight='length')
print(route)

CPU times: user 22.6 ms, sys: 0 ns, total: 22.6 ms
Wall time: 22.6 ms
[5629072001, 5629072000, 5629071975, 4239313191, 4239313081, 4239313075, 4239312638, 4239312409, 4239311933, 4239310797, 3189915006, 3175727789, 1263352467, 9283437397, 9283437392, 9283437388, 9283437390, 9283437375, 3210037368, 3185495471, 6689794785, 5919123188, 7309896051, 46411146, 3188608544, 46410382, 8238495242, 46409003, 46405684, 46402187, 5933448325, 4959039686, 46399571, 9407636382, 46392637, 9407636386, 7973142487, 8223730972, 1291587576, 1291587540, 46385544, 46383339, 46380741, 46379359, 46374887, 46374082, 3781170573, 7995426344, 3781170150, 3781170147, 3781170139, 2021897089, 2021897044, 7223158294, 7223158307, 2021897032, 7223210075, 7223210071, 1291468055, 3160068713, 3160068154, 46356661]


In [ ]:
route_map = ox.plot_route_folium(G_walk, route)
display(route_map)

In [ ]:
%%time 
nodes = pd.DataFrame.from_dict(dict(G_walk.nodes(data=True)), orient='index')

CPU times: user 123 ms, sys: 995 µs, total: 124 ms
Wall time: 131 ms


In [ ]:
%%time 
edges = nx.to_pandas_edgelist(G_walk)

C:\Users\joaquimg\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


CPU times: total: 2.45 s
Wall time: 2.51 s


In [ ]:
nodes.street_count.describe()

count   35211.000000
mean        2.816165
std         0.996772
min         1.000000
25%         3.000000
50%         3.000000
75%         3.000000
max         8.000000
Name: street_count, dtype: float64

In [ ]:
edges.length.describe()

count   99098.000000
mean       54.205131
std        68.289323
min         0.130000
25%        15.183000
50%        34.610000
75%        68.279000
max      1550.813000
Name: length, dtype: float64

In [ ]:
edges.loc[edges.length.idxmax()]

source                                              2632720004
target                                              4340833704
highway                                               tertiary
geometry     LINESTRING (5.0484437 52.4098672, 5.0492122 52...
est_width                                                  NaN
access                                                     NaN
maxspeed                                                   NaN
junction                                                   NaN
length                                             1550.813000
width                                                      NaN
oneway                                                   False
lanes                                                      NaN
osmid                                                  7039463
bridge                                                     NaN
ref                                                        NaN
service                                                

In [ ]:
%time longest = nx.shortest_path(G_walk,2632720004,46544942,weight='length')
print(longest)

CPU times: total: 0 ns
Wall time: 850 µs
[2632720004, 4868500506, 46526747, 3646762450, 46515220, 7896482872, 7896482772, 7896482536, 2876000370, 2876030674, 46525998, 2875982072, 2876049179, 2876046111, 2834850861, 3380450298, 3380450325, 3380450323, 1970783754, 1970783822, 1970783783, 1970783792, 1970789262, 1970785372, 46546263, 3478950226, 2869259996, 2876043898, 46544942]


In [ ]:
route_map = ox.plot_route_folium(G_walk, longest)
display(route_map)

# Dijkstra on steroids for road networks

In [ ]:
%%time
import pandana
network = pandana.Network(nodes['x'], nodes['y'], edges['source'], edges['target'], edges[['length']],twoway=True)

CPU times: total: 4.36 s
Wall time: 925 ms


In [ ]:
network.nodes_df.head()

,x,y
6316199,4.888396,52.370173
25596455,4.923563,52.364840
25596477,4.906097,52.367000
25645989,4.925075,52.365727
25645990,4.926110,52.366326


In [ ]:
network.edges_df.head()

,from,to,length
0,6316199,46379627,42.497000
1,6316199,46389218,225.577000
2,6316199,391355271,62.907000
3,25596455,8383889398,1.791000
4,25596455,46356773,41.700000


In [ ]:
df['pandana'] = network.get_node_ids(df.lon, df.lat).values
df

,address,color,city,country,lat,lon,osmnx,pandana
0,Centraal Station,blue,Amsterdam,NL,52.378901,4.900581,5629072001,5629071974
1,Amsterdam Business School,black,Amsterdam,NL,52.365107,4.911718,46356661,46356661
2,Artis,green,Amsterdam,NL,52.415605,4.837026,8116399477,8116399477
3,Arena,red,Amsterdam,NL,52.315990,4.942931,4622542635,2928936658
4,Ziggo Dome,purple,Amsterdam,NL,52.313790,4.938438,1925143759,4622542635


In [ ]:
%time path_pandana = network.shortest_path(df.iloc[2].pandana, df.iloc[3].pandana)

CPU times: total: 0 ns
Wall time: 2.99 ms


In [ ]:
%time path_nx = nx.shortest_path(G_walk,df.iloc[2].osmnx,df.iloc[3].osmnx,weight='length')

CPU times: total: 250 ms
Wall time: 260 ms


In [ ]:
A = set(path_pandana)
B = set(path_nx)
(A | B) - (A & B)

{46232158,
 46237572,
 46240581,
 46241169,
 46241205,
 46242940,
 46245940,
 46247806,
 46255649,
 46260704,
 46264996,
 46265577,
 46265994,
 46266419,
 46267226,
 46267229,
 46274385,
 46274670,
 46275510,
 46278192,
 46278488,
 46280478,
 46280842,
 46281209,
 46281524,
 46285636,
 46291556,
 46291719,
 46293298,
 46293761,
 46295375,
 46296981,
 46300577,
 46301759,
 46301760,
 46303077,
 46304102,
 46305144,
 46305209,
 46307985,
 46309621,
 46309912,
 46311730,
 46312415,
 46313094,
 46314564,
 46314956,
 46317329,
 46319331,
 46320807,
 46321546,
 46323267,
 46323943,
 46326231,
 46326678,
 46327739,
 46329521,
 46329829,
 46331717,
 46332009,
 46332010,
 46332803,
 46332984,
 46333825,
 46333911,
 46334874,
 46335169,
 46336020,
 46339231,
 93060999,
 252728564,
 254477712,
 262890789,
 295681891,
 295681899,
 369606077,
 452485084,
 497304704,
 497304720,
 878805275,
 878806012,
 878806213,
 878808875,
 926793900,
 1101933078,
 1101933080,
 1259406528,
 1333602328,
 136784414

In [ ]:
origs = [o for o in df.pandana for d in df.pandana]
dests = [d for o in df.pandana for d in df.pandana]
%time distances = network.shortest_path_lengths(origs, dests)

CPU times: total: 0 ns
Wall time: 7.55 ms


In [14]:
import numpy as np 

n = len(df)
pd.options.display.float_format = '{:.2f}'.format
pd.DataFrame(np.array(list(distances)).reshape(n,n),index=df.address,columns=df.address)

NameError: ignored

In [ ]:
np.random.seed(2021)
n = 500
sample = np.random.choice(np.array(network.nodes_df.index.values.tolist()), n, replace=False)
origs = [o for o in sample for d in sample]
dests = [d for o in sample for d in sample]

In [ ]:
%time distances = network.shortest_path_lengths(origs, dests)
%time table = pd.DataFrame(np.array(list(distances)).reshape(n,n),index=sample,columns=sample)

In [ ]:
departure = table.max(axis=1).idxmax()
arrival = table.loc[departure].idxmax()
%time path_pandana = network.shortest_path(departure, arrival)
%time path_nx = nx.shortest_path(G_walk,departure,arrival,weight='length')
A = set(path_pandana)
B = set(path_nx)
(A | B) - (A & B)

In [ ]:
%time paths = network.shortest_paths(origs,dests)

In [ ]:
sum(map(len,paths))

In [ ]:
for u,v in zip(paths[1][:-1],paths[1][1:]):
    print(G_walk.get_edge_data(u,v)[0].get('name',''))

In [ ]:
route_map = ox.plot_route_folium(G_walk, paths[1],color='red',map=Map)
display(route_map)